In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from snownlp import SnowNLP

# 广播剧 sound_id 列表
#xxxxxx替换为所示链接的数字，可提供多个idhttps://www.missevan.com/sound/player?id=11105876
sound_ids = ['xxxxxx'] 

def fetch_danmaku(sound_id):
    url = f"https://www.missevan.com/sound/getdm?soundid={sound_id}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    return response.text if response.status_code == 200 else None

def parse_danmaku(xml_content, sound_id):
    root = ET.fromstring(xml_content)
    result = []
    for d in root.findall('d'):
        p = d.get('p').split(',')
        time = float(p[0])
        uid = p[6]
        text = d.text
        if text:
            try:
                sentiment = SnowNLP(text).sentiments
                result.append({'sound_id': sound_id, 'time': time, 'uid': uid, 'content': text, 'sentiment': sentiment})
            except:
                continue
    return result

# 整合所有弹幕
all_danmakus = []
for sid in sound_ids:
    xml_data = fetch_danmaku(sid)
    if xml_data:
        all_danmakus.extend(parse_danmaku(xml_data, sid))

df = pd.DataFrame(all_danmakus)
# 整合所有弹幕
all_danmakus = []
for sid in sound_ids:
    xml_data = fetch_danmaku(sid)
    if xml_data:
        all_danmakus.extend(parse_danmaku(xml_data, sid))

df = pd.DataFrame(all_danmakus)
df.to_csv(r'D:/danmakus.csv', index=False, encoding='utf-8-sig') #自行选择存储地址
print("爬取完成，共获取弹幕数：", len(df))